In [14]:
 # cnn model trained on CIFAR-10 dataset

In [15]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader

import torch.nn.functional as F

In [16]:
# transformations
composed = transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                        ])

In [17]:
# CIFAR-10 dataset
train_data = torchvision.datasets.CIFAR10(root='./data',train=True,
                                          download=True,transform=composed)

test_data = torchvision.datasets.CIFAR10(root='./data',train=False,
                                         download=True,transform=composed)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
# hyperparameters
batch_size = 4
num_epochs = 4
learning_rate = 0.01

# device
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [19]:
# DataLoaders

train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=2)

In [20]:
len(train_loader),len(test_loader)

(12500, 2500)

In [21]:
# label classes
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

num_classes = len(classes)
print(num_classes)

10


In [25]:
# model class
class ConvNet(nn.Module):
    def __init__(self,num_classes):
        super(ConvNet,self).__init__()
        
        # cnn layers
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)
        
        #fc layers
        self.linear1 = nn.Linear(in_features=16*5*5,out_features=120)
        self.linear2 = nn.Linear(in_features=120,out_features=84)
        self.linear3 = nn.Linear(in_features=84,out_features=num_classes)

    def forward(self,x):
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x1)))
        x2_flat = x2.view(-1,16*5*5) # Flatten
        lin1 = self.linear1(x2_flat)
        x3 = F.relu(lin1)
        x4 = F.relu(self.linear2(x3))
        x5 = F.relu(self.linear3(x4))
        return lin1,x5

In [26]:
# model defining
model = ConvNet(num_classes)
model = model.to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [27]:
n_total_steps = len(train_loader)

# training loop
for epoch in range(num_epochs):
    for ii,(images,labels) in enumerate(train_loader):
        # original shape = [4,3,32,32] : 4 := batch_size, 3 := RGB, dims =32x32
        # input_layer = 3 input_channels, 6 output_channels, 5 kernel_size
        images = images.to(device)
        labels = labels.to(device)

        # forward
        _,outputs = model(images)
        loss = criterion(outputs,labels)

        # backward and updates
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (ii+1)%2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{ii+1}/{n_total_steps}], Loss = {loss.item():.6f}')

    print('----------------------------------------')

Epoch [1/4], Step [2000/12500], Loss = 2.291288
Epoch [1/4], Step [4000/12500], Loss = 1.638216
Epoch [1/4], Step [6000/12500], Loss = 1.212903
Epoch [1/4], Step [8000/12500], Loss = 1.595376
Epoch [1/4], Step [10000/12500], Loss = 1.581806
Epoch [1/4], Step [12000/12500], Loss = 2.580363
----------------------------------------
Epoch [2/4], Step [2000/12500], Loss = 1.167568
Epoch [2/4], Step [4000/12500], Loss = 1.759755
Epoch [2/4], Step [6000/12500], Loss = 1.679082
Epoch [2/4], Step [8000/12500], Loss = 2.505718
Epoch [2/4], Step [10000/12500], Loss = 1.020406
Epoch [2/4], Step [12000/12500], Loss = 0.843665
----------------------------------------
Epoch [3/4], Step [2000/12500], Loss = 1.695767
Epoch [3/4], Step [4000/12500], Loss = 1.212294
Epoch [3/4], Step [6000/12500], Loss = 0.734185
Epoch [3/4], Step [8000/12500], Loss = 1.875462
Epoch [3/4], Step [10000/12500], Loss = 0.649675
Epoch [3/4], Step [12000/12500], Loss = 0.096931
----------------------------------------
Epoch [

In [28]:
# evaluating model and getting features of every image
features = []

with torch.no_grad():
    # for entire test set
    n_correct = 0
    n_samples = 0

    # for each class label
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]

    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        ftrs,outputs = model(images)
        features.append(ftrs)

        _,preds = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (preds == labels).sum().item()

        # iterating over the batch 
        for i in range(batch_size):
            label = labels[i]
            pred = preds[i]
            n_class_samples[label] +=1
            if(label == pred):
                n_class_correct[label] += 1
            
    accuracy = n_correct/float(n_samples)
    print(f'Accuracy of model on test set = {accuracy:.4f}')
    
    print('-------------------------------------------')
    
    # printing accuracy per class
    for i in range(num_classes):
        accuracy = n_class_correct[i]/float(n_class_samples[i])
        print(f'Accuracy of {classes[i]} : {accuracy}')

Accuracy of model on test set = 0.5883
-------------------------------------------
Accuracy of plane : 0.53
Accuracy of car : 0.775
Accuracy of bird : 0.33
Accuracy of cat : 0.371
Accuracy of deer : 0.493
Accuracy of dog : 0.683
Accuracy of frog : 0.659
Accuracy of horse : 0.604
Accuracy of ship : 0.66
Accuracy of truck : 0.778


In [51]:
model

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=400, out_features=120, bias=True)
  (linear2): Linear(in_features=120, out_features=84, bias=True)
  (linear3): Linear(in_features=84, out_features=10, bias=True)
)

In [37]:
len(features), features[0].shape

(2500, torch.Size([4, 120]))

In [40]:
# convert tensor to numpy
for i in range(len(features)):
    features[i] = features[i].cpu().numpy()

type(features[0])

numpy.ndarray

In [42]:
features = np.array(features)

In [45]:
# reshaping (2500,4,120) into (2500x4,120) as we had taken batch_size=4 during loading
dims = features.shape
features = features.reshape(dims[0]*dims[1],dims[2])

features.shape

(10000, 120)

In [ ]:
# convert to dataframe
import pandas as pd

ftrs_df = pd.DataFrame(features)

In [48]:
# convert df to csv
ftrs_df.to_csv('./hopefully_this_is_desired_output.csv',index=False)

In [49]:
# reloading the saved csv into a df

ftrs_csv_df = pd.read_csv('./hopefully_this_is_desired_output.csv')
ftrs_csv_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119
0,-4.041168,-4.620143,-1.781208,-0.723168,-3.685660,-2.465509,-1.312625,-3.474266,-0.666465,-6.764115,-1.050546,-1.225386,-5.585804,-6.634646,-1.438204,-2.241868,7.081588,2.125396,-1.947522,-0.275793,3.568234,-1.743907,0.924543,-3.827226,-5.649690,-0.910609,-3.933114,0.920461,-1.389764,-5.669378,-4.391663,-5.216137,-0.258783,-0.894288,-0.410219,-1.349462,1.067147,-1.355117,-1.064318,-3.026964,...,-2.290328,-3.284536,-0.969044,-1.620327,-0.119449,-0.457821,-8.265476,-2.297562,-4.014337,1.185151,1.269268,-1.117636,-2.991186,2.151464,-1.947825,-3.641346,-2.994073,-0.979270,-2.316109,2.873752,0.108960,-1.364055,1.481271,0.531562,0.323296,-5.919993,-5.442319,1.533954,3.005888,-1.049830,-3.512131,-0.805302,-1.204902,-5.971713,-0.976589,1.009387,1.615299,3.179398,-4.912114,-2.614215
1,-9.957033,-7.765330,-1.790383,-0.513650,-7.011168,-6.452775,-4.808393,1.504367,5.637000,-5.802796,-0.197615,-4.988310,-0.849823,-14.711400,-3.160456,-3.262569,-2.373764,2.796687,-4.199276,9.975645,-7.214971,-12.988875,-0.164072,1.181543,-3.065121,-2.355663,-0.256631,2.123674,-14.889133,-13.027492,-10.614709,-8.351180,-1.433402,-3.776729,-0.669025,-2.112906,-1.546241,-1.212393,-2.231576,2.277348,...,-8.920451,-3.659581,-5.063305,-1.569890,2.184623,-4.130387,-5.537583,-4.430237,-3.639200,-6.295833,-3.998492,-3.174933,-2.193366,-2.362017,-12.756830,-3.013829,1.958126,0.675121,-2.301796,-6.873603,1.879667,0.138329,-3.954981,-4.719856,-7.409155,4.867367,4.480630,-3.505374,-0.838354,-2.209542,-2.923599,1.004017,-3.588692,1.592608,-2.782061,-10.455246,-1.201422,-3.773453,8.664276,2.582627
2,-3.259100,-4.725335,-1.732987,-0.350862,-2.510638,-1.016953,-2.638103,0.490408,1.988256,-1.694846,-3.096874,-1.890489,3.303818,-4.203105,-1.144866,-1.718373,-1.212967,-1.540200,-2.000094,6.209831,-2.407688,-2.482808,-3.420059,-2.518446,-1.848594,-0.182664,0.208687,-1.152272,-6.492965,-5.474305,-6.623555,-1.836562,-0.957338,-3.144039,1.365988,-0.301060,-0.020419,0.130050,0.920070,1.425205,...,-4.182159,0.190383,-1.756984,-1.119064,3.226987,-0.699961,-3.431693,-2.847187,-0.579006,-5.512690,-0.847085,-0.234986,-2.013248,1.597455,-3.099414,-0.507322,0.702549,-1.291992,-2.835172,-2.163636,-2.189157,-0.457645,-0.951948,-3.104322,-1.511662,2.800468,2.530758,-1.824971,1.888536,-2.243031,1.017182,-0.313621,-1.285848,0.224540,0.594916,-6.662509,-4.193942,-3.578846,2.050025,4.811572
3,-6.947447,-2.192886,-2.688921,-0.302683,-3.348802,-7.758157,-1.579127,-0.364822,2.110183,-1.836274,4.686403,-2.857338,1.145200,-5.265627,-1.213157,-2.253142,0.193063,1.295935,-1.287893,-1.857092,-1.810665,-6.985744,-0.930199,1.945227,-4.829821,-0.669064,1.183280,2.773684,-9.733597,-6.384218,-5.258634,-4.056843,-2.712468,-0.336427,2.897691,-1.555371,-1.226870,-1.126374,-0.880810,2.099252,...,-4.290386,1.363143,-4.668521,-1.648762,-1.301664,0.492377,-3.948330,-2.044114,-1.675054,-4.437402,-2.133241,-4.287119,-1.410782,0.469959,-6.563445,2.793151,-2.305392,0.656179,-3.242972,-2.408987,-3.219388,-1.459225,-3.233110,-0.403408,-6.117755,1.224759,-2.827114,-2.127799,0.522055,-0.697712,-4.167525,0.337876,-5.808713,0.223810,-1.281335,-7.923990,3.113291,-3.527139,5.438856,-4.269650
4,2.468882,-6.016874,-1.166926,-0.915221,-1.880585,-3.809862,-0.337698,0.184816,-2.649029,0.869480,-2.532718,0.269207,-0.775140,-3.027358,-1.015006,-1.981597,1.547824,-6.011969,-2.795195,-2.756511,-0.840845,2.145902,-3.571432,-3.109736,-2.124403,2.103023,-0.911387,-0.886674,2.154503,-1.383355,3.116836,-3.671776,3.616449,3.479704,1.338850,1.070671,2.403682,-0.434449,-3.212668,-1.211945,...,-2.965212,-1.148267,-0.040053,-1.612430,-0.574172,-1.335602,1.404068,-2.787293,-0.125314,-2.123843,1.292274,-1.485380,-2.191170,0.010183,0.420879,-2.250148,-6.829504,-2.711986,-1.752280,-2.156443,-6.32

In [50]:
# saving model
MODEL_PATH = 'saved_model.pth'
torch.save(model.state_dict(),MODEL_PATH)

In [ ]:
# # printing parameters of the model

# for param in model.parameters():
#     print(param.shape)
#     print(str(param))